In [1]:
%matplotlib inline

In [2]:
from sklearn import linear_model
import matplotlib.pyplot as plt
import time
import Xlib.display as display
import gtk, pygtk
import csv
import numpy as np
import ConfigParser
import tensorflow as tf

In [3]:
theta = 3.5e-07

In [4]:
#read write project path
#TODO: configure this file according to your path: BehavioralAuthentication/src/main/php/configuration.cfg
config = ConfigParser.RawConfigParser()
config.read('configuration.cfg')

#project_dir_path = config.get('Section1', 'project_dir')
#resource_path = project_dir_path + "src/test/resources/"

[]

In [5]:
window = gtk.Window()
screen = window.get_screen()

x_size = screen.get_width()
y_size = screen.get_height()

In [6]:
#make new

#ted dunning model

def extractFeatures(filename, n_last_timestamps_as_features = 200, normalize = True, bias = True):
    queue = {}
    
    diffTime = {}
    oldTime = -1.0
    
    maxX = 0.0
    
    x_width = float(screen.get_width()) 
    y_width = float(screen.get_height()) 
    
    
    

    x_attribute = -1
    y_attribute = -1
    i = 0
    r = 0
    
    record_number = len(list(csv.reader(open(filename))))
    print record_number
    
    with open(filename, 'r') as csvfile:
         spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')   
            
         for row in spamreader:
            
            feature_num = len(row)
            
            record_size = feature_num * n_last_timestamps_as_features
            data_size = record_number - n_last_timestamps_as_features - 1
            
            if (i==0):
                
                data = np.zeros((data_size, record_size))
                target = np.zeros((data_size, feature_num))
                
                #find mouse position coordinate columns in the file
                for a in range(feature_num):
                    if (row[a] == "mouse_X"):
                        x_attribute = a
                    if (row[a] == "mouse_Y"):
                        y_attribute = a
                        
            else:
                #normalize mouse coordinates
                if (normalize):
                     row[x_attribute] = float(row[x_attribute]) / x_width   
                     row[y_attribute] = float(row[y_attribute]) / y_width
                
                
                if (i <= n_last_timestamps_as_features):
                    data [0,(i-1)*feature_num : i*feature_num] = row
                    
                elif(i == n_last_timestamps_as_features + 1):
                    target[0,:] = row
                    
                    r = r + 1
                    
                    
                else:
                    data[r, 0 : (n_last_timestamps_as_features - 1) * feature_num] = \
                        data[(r - 1), feature_num : (n_last_timestamps_as_features) * feature_num]
                    data[r, (n_last_timestamps_as_features - 1) * feature_num : (n_last_timestamps_as_features) * feature_num] = \
                        target[(r-1),:]
                        
                    target[r,:] = row                   
                
                    r = r + 1
            i = i + 1
            
    if (bias):
        data = np.hstack((data, np.ones((data.shape[0], 1), dtype=data.dtype)))
    
    
    print("original shape: " + str(data.shape))

    return ((data, target))
            
            
 

In [7]:
extractFeatures("/home/felix/mousetracking/BehavioralAuthentication/src/main/java/features4.txt", \
                n_last_timestamps_as_features = 1, normalize = False, bias = True)

9
original shape: (7, 3)


(array([[  1.,   2.,   1.],
        [  3.,   4.,   1.],
        [  5.,   6.,   1.],
        [  7.,   8.,   1.],
        [  9.,  10.,   1.],
        [ 11.,  12.,   1.],
        [ 13.,  14.,   1.]]), array([[  3.,   4.],
        [  5.,   6.],
        [  7.,   8.],
        [  9.,  10.],
        [ 11.,  12.],
        [ 13.,  14.],
        [ 15.,  16.]]))

In [8]:
def generateModelData((data1,target1), n_last_timestamps_as_features = 200, 
                      v = False, build_all_model = True, p_train_general = 1.0, 
                      stddev_factor = 1.0, returnSmoothed = False):
    
    N_total = data1.shape[0]
    N_total_test = N_total * 0.4
    
    #(data,target) = getMovementsOnly(data1, target1)
    (data,target) = (data1, target1)
    N = data.shape[0]
    
    print data.shape
    print target.shape
    
    p_train = 0.6
    X_train = data[0:(N * p_train_general * p_train),:]
    y_train = target[0:(N * p_train_general * p_train),:]
    X_test = data[((N * p_train) + n_last_timestamps_as_features):N,:]
    y_test = target[((N * p_train) + n_last_timestamps_as_features):N,:]
    
    N_total_test = X_test.shape[0]
    

    regr = linear_model.LinearRegression(normalize=False)
    
    dist_train = getDistance(y_train)
    
    #impossible threshold
    impossible_threshold = np.array([stddev_factor, stddev_factor])


    # Train the model using the training sets
    regr.fit(X_train, y_train);
    
    #training error
    dist_train = getDistance(y_train)
    
    prediction_train = regr.predict(X_train)
    
    #target y is constrained to be within in [0,1]
    prediction_train[prediction_train < 0.0] = 0
    prediction_train[prediction_train > 1.0] = 1
    
    diff_train = np.abs(prediction_train - y_train)
    diff_x_train = diff_train[:,0]
    diff_y_train = diff_train[:,1]
    
    train_error_mse = (np.sum((np.power(diff_x_train, 2) + np.power(diff_y_train, 2))) / diff_x_train.shape[0])
    train_error_dist = (np.sum((np.power(diff_x_train, 2) + np.power(diff_y_train, 2))) / dist_train)
    
    (smoothed_time_error_train, smoothed_distance_error_train, N_smoothed_train) = \
    get_smoothed_result(X_train,y_train, diff_train, impossible_threshold, v = False) 
   

    #test error
    dist_test = getDistance(y_test)
    
    prediction_test = regr.predict(X_test)
    
    #target y is constrained to be within in [0,1]
    prediction_test[prediction_test < 0.0] = 0
    prediction_test[prediction_test > 1.0] = 1
    
    diff_test = np.abs(prediction_test - y_test)        
    diff_x_test = diff_test[:,0]
    diff_y_test = diff_test[:,1]
    
    test_error_mse = (np.sum(np.power(diff_x_test, 2) + np.power(diff_y_test, 2)) / diff_x_test.shape[0])
    test_error_dist = (np.sum(np.power(diff_x_test, 2) + np.power(diff_y_test, 2)) / dist_test)
    
    (smoothed_time_error_test, smoothed_distance_error_test, N_smoothed) = \
    get_smoothed_result(X_test,y_test, diff_test, impossible_threshold, v = v) 
    
    if (v):
        
        print("threshold: " + str(impossible_threshold))
           
        #Prediction of x-coordinate of the cursor over time
        plt.plot(y_test[:,0])
        plt.plot(prediction_test[:,0])
        plt.ylabel('relative postion - x coordinate of the cursor')
        plt.title("regression - x")
        plt.show()
        
        #Prediction of y-coordinate of the cursor over time
        plt.plot(y_test[:,1])
        plt.plot(prediction_test[:,1])
        plt.ylabel('relative postion - y coordinate of the cursor')
        plt.title("regression - y")
        plt.show()
        
        print("test - MSE: %.10f" % test_error_mse)
        print("test - MSE - dist: %.10f" % test_error_dist)
        
        print ("test - smoothed MSE: " + str(smoothed_time_error_test))
        print ("test - smoothed MSE per dist: " + str(smoothed_distance_error_test))
        
        #Test error over time
        threshold_line = np.ones(diff_x_test.shape) * impossible_threshold[0]        
        plt.plot(diff_x_test)
        plt.plot(threshold_line)
        plt.ylabel('Test error')
        plt.title("ErrorPerTestTime - x")
        plt.show()
        threshold_line = np.ones(diff_y_test.shape) * impossible_threshold[1]        
        plt.plot(diff_y_test)
        plt.plot(threshold_line)
        plt.ylabel('Test error')
        plt.title("ErrorPerTestTime - y")
        plt.show()
        
        #Training error over time
        threshold_line = np.ones(diff_x_train.shape) * impossible_threshold[0]
        plt.plot(diff_x_train)
        plt.plot(threshold_line)
        plt.ylabel('Training error')
        plt.title("ErrorPerTrainTime - x")
        plt.show()
        
        threshold_line = np.ones(diff_y_train.shape) * impossible_threshold[1]
        plt.plot(diff_y_train)
        plt.plot(threshold_line)
        plt.ylabel('Training error')
        plt.title("ErrorPerTrainTime - y")
        plt.show()
        
        print("train - MSE: %.10f" % train_error_mse)
        print("train - MSE - dist: %.10f" % train_error_dist)    
        
        print ("train - smoothed MSE: " + str(smoothed_time_error_train))
        print ("train - smoothed MSE per dist: " + str(smoothed_distance_error_train))
    
        
    if (returnSmoothed):
        return((smoothed_time_error_test, smoothed_distance_error_test, float(N_smoothed) / float(N_total_test)))
    
    if (build_all_model):   
        final_regr = linear_model.LinearRegression(normalize=True)
    
        final_regr.fit(data, target);
    
        return ((final_regr, impossible_threshold))
        #return (regr, two_sigma)
    else:
        return test_error_mse

In [9]:
def generateModel(dataFile, n_last_timestamps_as_features = 200, v = False, stddev_factor = 0.0):
    (data,target) = extractFeatures(dataFile, n_last_timestamps_as_features)
    
    return generateModelData((data,target), n_last_timestamps_as_features = n_last_timestamps_as_features, 
                             v = v, stddev_factor = stddev_factor)

In [10]:
def getMovementsOnly(data, target):
    moveOnlyData = []
    moveOnlyTarget = []
    
    first = True
    
    for l in range(data.shape[0]):
        if (first):
            
            #print data[l]
            
            moveOnlyData = [data[l]]
            moveOnlyTarget = [target[l]]
            first = False
        else:
            
            #print ("new: " + str(data[l]))
            #print ("old: " + str(data[l]))
            
            if (np.any(data[l] != data[l-1]) or np.any(target[l] != target[l-1])):
                moveOnlyData = np.concatenate((moveOnlyData, [data[l]]), axis=0)
                moveOnlyTarget = np.concatenate((moveOnlyTarget, [target[l]]), axis=0)

    return (np.matrix(moveOnlyData), np.matrix(moveOnlyTarget))
            

In [11]:
def euclidean (p1, p2):
    return np.sqrt(np.sum(np.power(p1 - p2, 2)))

def getDistance(target):
    distance = 0.0
    for t in np.arange(1, target.shape[0], 1):
        distance += euclidean(target[t,:], target[(t-1),:])
    
    return distance    

def getSmoothedDistance(target, isOverThreshold):
    
    distance = 0.0
    for t in np.arange(1, target.shape[0], 1):
        last_point_x = target[t-1,0]
        last_point_y = target[t-1,1]
        if (isOverThreshold[t] == False):
            distance += euclidean(target[t,:], np.array([last_point_x,last_point_y])) 
    
    return distance

In [12]:
def get_smoothed_result(newData, newTarget, diff, threshold, v = False):
    
    diff_smoothed = np.copy(diff)
    
    N_smoothed = newTarget.shape[0]
    
    is_over_threshold = np.logical_or((diff_smoothed[:,0] > threshold[0]),(diff_smoothed[:,1] > threshold[1]))
    
    newdist = getSmoothedDistance(newTarget, is_over_threshold)
    
    for l in range(diff_smoothed.shape[0]):
        if (is_over_threshold[l]):
            diff_smoothed[l,0] = 0
            diff_smoothed[l,1] = 0
            N_smoothed -= 1
            
    if (v):
        plt.plot(diff_smoothed[:,0])
        plt.ylabel('prediction x error -smoothed')
        plt.show()

        plt.plot(diff_smoothed[:,1])
        plt.ylabel('prediction y error -smoothed')
        plt.show()
    
    time_error = 0.0
    if (N_smoothed > 0.0):
        time_error = np.sum(np.power(diff_smoothed,2)) / N_smoothed
    
    distance_error = 0.0
    if (newdist > 0.0):
        distance_error = np.sum(np.power(diff_smoothed,2)) / newdist
    
    return ((time_error,distance_error, N_smoothed))

In [13]:
def showStats(file, m, threshold, n_last_timestamps_as_features = 200, normalize = True, v = True):
    (fileX1, fileY1) = extractFeatures(file, 
                                     n_last_timestamps_as_features = n_last_timestamps_as_features, 
                                     normalize = normalize) 
    
    return showStatsData((fileX1, fileY1), m, threshold, n_last_timestamps_as_features = n_last_timestamps_as_features, v = v)

In [14]:
t = 1000000.0
    
for i in range(1000001):
    t += 0.000001

t -= 1000000.0

print t

1.0000086145


In [15]:
def showStatsData((fileX, fileY), m, threshold, n_last_timestamps_as_features = 200, v = True):
    
    (newData, newTarget) = getMovementsOnly(fileX, fileY)   
    
    if newData.shape[0] <= 1: return(0.0, 0.0, 0, 1)
    
    #distance
    dist = getDistance(newTarget)
    
    prediction_sample = m.predict(newData)
    
    #target y is constrained to be within in [0,1]
    prediction_sample[prediction_sample < 0.0] = 0
    prediction_sample[prediction_sample > 1.0] = 1

    ErrorPerTestTime = np.sum(np.absolute(prediction_sample - newTarget),1);
    
    threshold_line = np.ones(ErrorPerTestTime.shape) * (threshold[0] + threshold[1])
    
    if (v):
        #error of sample
        plt.plot(ErrorPerTestTime)
        plt.plot(threshold_line)
        plt.ylabel('prediction error')
        plt.show()

        #Prediction of x-coordinate of the cursor over time
        plt.plot(prediction_sample[:,0])
        plt.plot(newData[:,0])
        plt.ylabel('relative x-coordinate')
        plt.title("Regression for x-coordinate")    
        plt.show()

        #Prediction of y-coordinate of the cursor over time
        plt.plot(prediction_sample[:,1])
        plt.plot(newData[:,1])
        plt.ylabel('relative y-coordinate')
        plt.title("Regression for y-coordinate")
        plt.show()

    diff = np.absolute(prediction_sample - newTarget)
    
    N = newTarget.shape[0]
    
    if (v):
        print ("MSE per 0.03s: " + str(np.sum(np.power(diff,2)) / N))
        print ("MSE per pixel move: " + str(np.sum(np.power(diff,2)) / dist))
        print ("number of records: " + str(N))

        print ("\nmax - |x+y|: " + str(np.max(diff)))
        print ("max - squared: " + str(np.max(np.power(diff,2))))
    
    (time_error, distance_error, N_smoothed) = get_smoothed_result(newData, newTarget, diff, threshold)    
    
    if (v):
        print ("smoothed MSE: " + str(time_error))
        print ("smoothed MSE per dist: " + str(distance_error))
    
    if (time_error > theta): 
        if (v):
            print ("This is not Felix - You are unauthorized!")
        return (time_error, distance_error, N_smoothed, 0)
    else:
        if (v):
            print ("This is Felix - Welcome to this awesome computer!")  
        return (time_error, distance_error, N_smoothed, 1)

In [53]:
p_train = 0.6
p_valid = 0.2
#p_test = 0.2

rr_steps = 100

(data,target) = extractFeatures("/home/felix/mousetracking/BehavioralAuthentication/src/main/java/features3.txt", \
                                rr_steps)

print(target.shape)

data = np.matrix(data, dtype="float32")
target = np.matrix(target, dtype="float32")

N = data.shape[0]

train_dataset = data[0:(N * p_train),:]
train_labels = target[0:(N * p_train),:]
valid_dataset = data[((N * p_train) + rr_steps):((N * (p_train + p_valid))),:]
valid_labels = target[((N * p_train) + rr_steps):((N * (p_train + p_valid))),:]
test_dataset = data[((N * (p_train + p_valid)) + rr_steps):N,:]
test_labels = target[((N * (p_train + p_valid)) + rr_steps):N,:]

(train_dataset,train_labels) = getMovementsOnly(train_dataset, train_labels)

print(train_dataset.shape)
print(train_labels.shape)

num_features = train_dataset.shape[1]
num_labels = train_labels.shape[1]

186051
original shape: (185950, 201)
(185950, 2)
(25126, 201)
(25126, 2)


In [54]:
def accuracy(predictions, labels):
  
  return np.sum((np.square(np.matrix(predictions) - np.matrix(labels))).mean(axis=0))

In [55]:
batch_size = 128
hidden_layers = 1224
beta = 5e-4 #l2 regularization parameter

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, num_features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([num_features, hidden_layers], seed=1))
  b1 = tf.Variable(tf.zeros([hidden_layers]))
    
  w2 = tf.Variable(
    tf.truncated_normal([hidden_layers, num_labels], seed=2))
  b2 = tf.Variable(tf.zeros([num_labels]))   
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), w2) + b2 

  loss = tf.reduce_mean(tf.square(tf.nn.softmax(logits) - tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) +
                  tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2))
  # Add the regularization term to the loss.
  loss += beta * regularizers
  
  # Optimizer.
  batch = tf.Variable(0)

  learning_rate = tf.train.exponential_decay(
    0.01,                # Base learning rate.
    batch * batch_size,  # Current index into the dataset.
    train_dataset.shape[0],          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
  # Use simple momentum for the optimization.
  optimizer = tf.train.MomentumOptimizer(learning_rate,
                                     0.9).minimize(loss,
                                                   global_step=batch) 


  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax( tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2 )
    
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2 )

In [58]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %f" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %f" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %f" % accuracy(test_prediction.eval(), test_labels))
  
  rmse = np.sqrt((np.square(test_prediction.eval() - test_labels)).mean(axis=0))

Initialized
Minibatch loss at step 0: 48.467491
Minibatch accuracy: 0.947044
Validation accuracy: 0.853530
Minibatch loss at step 500: 46.141380
Minibatch accuracy: 0.710467
Validation accuracy: 0.483996
Minibatch loss at step 1000: 44.189671
Minibatch accuracy: 0.599737
Validation accuracy: 0.483845
Minibatch loss at step 1500: 42.412067
Minibatch accuracy: 0.232574
Validation accuracy: 0.483634
Minibatch loss at step 2000: 41.186825
Minibatch accuracy: 0.490554
Validation accuracy: 0.482520
Minibatch loss at step 2500: 40.269745
Minibatch accuracy: 0.958225
Validation accuracy: 0.481822
Minibatch loss at step 3000: 39.093845
Minibatch accuracy: 0.574267
Validation accuracy: 0.481039
Test accuracy: 0.449231


In [59]:
print rmse

[[ 0.47994742  0.4678472 ]]


In [ ]:
from tensorflow.python.ops.constant_op import constant
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np

'''
To classify images using a bidirectional reccurent neural network, we consider every image row as a sequence of pixels.
Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input =  num_features# MNIST data input (img shape: 28*28)
n_steps = rr_steps # timesteps
n_hidden = hidden_layers # hidden layer num of features
n_classes = num_labels # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
# Tensorflow LSTM cell requires 2x n_hidden length (state & cell)
istate_fw = tf.placeholder("float", [None, 2*n_hidden])
istate_bw = tf.placeholder("float", [None, 2*n_hidden])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'hidden': tf.Variable(tf.random_normal([n_input, 2*n_hidden])),
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([2*n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def BiRNN(_X, _istate_fw, _istate_bw, _weights, _biases):

    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs = rnn.bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, _X,
                                             initial_state_fw=_istate_fw,
                                             initial_state_bw=_istate_bw)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

pred = BiRNN(x, istate_fw, istate_bw, weights, biases)

cost = tf.reduce_mean(tf.square(tf.nn.softmax(pred) - y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                       istate_fw: np.zeros((batch_size, 2*n_hidden)),
                                       istate_bw: np.zeros((batch_size, 2*n_hidden))})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys,
                                                istate_fw: np.zeros((batch_size, 2*n_hidden)),
                                                istate_bw: np.zeros((batch_size, 2*n_hidden))})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,
                                             istate_fw: np.zeros((batch_size, 2*n_hidden)),
                                             istate_bw: np.zeros((batch_size, 2*n_hidden))})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                                             istate_fw: np.zeros((test_len, 2*n_hidden)),
                                                             istate_bw: np.zeros((test_len, 2*n_hidden))})